In [1]:
import pandas as pd

In [3]:
data_path = '../../handson-gb-main/Chapter06/heart_disease.csv'
df = pd.read_csv(data_path)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


### 6.1.2 XGBClassifier

In [5]:
from xgboost import XGBClassifier 
from sklearn.metrics import accuracy_score 

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [6]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

'gbtree' 부스터는 그레이디언트 부스팅 트리  
'binary:logistic' 목적 함수는 이진 분류의 기본 손실 함수  
XGBClassifier는 이 두 설정이 기본값이지만 이후 장에서 이 값을 바꿀 때 익숙해지도록 사용해 봄. 

In [7]:
from sklearn.model_selection import cross_val_score 
import numpy as np 
scores = cross_val_score(model, X, y, cv=5)
print('정확도: ', np.round(scores, 2))
print(f'정확도 평균: {scores.mean():.2f}')

정확도:  [0.84 0.85 0.82 0.8  0.77]
정확도 평균: 0.81


### 6.1.3 StratifiedKFold
교차 검증과 그리드 서치에서 분류 모델을 사용할 때 기본적으로 StratifiedKFold를 사용하여 폴드를 나눈다.  
회귀 모델을 사용할 때는 KFold 클래스를 사용한다.  
계층별 분할기(stratified splitter)인 StratifiedKFold는 각 폴드의 타깃 레이블의 비율을 동일하게 만든다. 

In [8]:
from sklearn.model_selection import StratifiedKFold 
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2) # shuffle 지정 시 데이터를 나누기 전에 섞는다. 
# kfold 객체를 cross_val_score(), GridSearchCV, RandomizedSearchCV에 전달하여 폴드를 나누는 데 사용할 수 있다. 

In [9]:
scores = cross_val_score(model, X, y, cv=kfold)
print('정확도: ', np.round(scores, 2))
print(f'정확도 평균: {scores.mean():.2f}')

정확도:  [0.72 0.82 0.75 0.8  0.82]
정확도 평균: 0.78


### 6.1.5 GridSearchCV와 RandomizedSearchCV 사용하기

In [10]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [15]:
def grid_search(params, random=False):
    xgb = XGBClassifier(booster='gbtree', objective='binary:logistic',
                       random_state=2, verbosity=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, n_jobs=-1, random_state=2)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    
    grid.fit(X, y)
    
    best_params = grid.best_params_
    print("최상의 매개변수: ", best_params)
    best_score = grid.best_score_
    print(f"최상의 점수: {best_score:.5f}")

## 6.2 XGBoost 하이퍼파라미터 튜닝
- n_estimators : 앙상블의 트리 개수, 잔차에 훈련되는 트리 개수이다.
- learning_rate : 부스팅 각 단계에서 트리의 기여도를 감소시킨다. 과대적합 방지(기본값 : 0.3)
- max_depth : 분할 횟수에 해당하는 트리의 깊이를 결정, 과대적합 방지(기본값 : 6)
- gamma : 노드 분할을 위한 최소 손실 감소를 지정, gamma를 증가시키면 보수적인 모델이 만들어진다. 
- min_child_weight : 노드를 분할하기 위해 필요한 최소 가중치 합, 샘플 가중치 합이 이 값보다 작으면 더이상 분할하지 않는다. 과대적합 방지
- subsample : 각 부스팅 단계에서 사용되는 훈련 샘플의 비율을 제한, 과대적합 방지
- colsample_bytree : 각 부스팅 단계마다 사용할 특성의 비율을 제한. 특성의 영향을 제한하고 분산을 줄이는 데 유용하다. 과대적합 방지
- colsample_bylevel : 트리 깊이마다 사용할 특성의 비율을 제한. 
- colsample_bynode : 노드를 분할할 때마다 사용할 특성의 비율을 제한

In [17]:
grid_search(params={'n_estimators':[100, 200, 400, 800]})

최상의 매개변수:  {'n_estimators': 100}
최상의 점수: 0.78235


In [18]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

최상의 매개변수:  {'learning_rate': 0.05}
최상의 점수: 0.79585


## 6.3 조기 종료 적용
반복적인 머신러닝 알고리즘의 훈련 횟수를 제한한다.  
사전에 정한 훈련 반복 횟수를 채우지 않더라도 연속적인 n번의 반복횟수 동안에 모델이 향상되지 않으면 훈련을 중지한다.  
- early_stopping_rounds : n_estimator 최적화
- eval_set : 평가할 데이터셋을 지정. 예를 들면 X_test, y_test
- eval_metric : 측정 방식을 지정. 일반적으로 분류는 'error', 회귀는 'rmse'

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [27]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic', eval_metric='error')
eval_set = [(X_test, y_test)]

model.fit(X_train, y_train, eval_set=eval_set)

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation_0-error:0.18421
[30]	validation_0-error:0.18421
[31]	validation_0-

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='error', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [28]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'정확도 : {accuracy * 100:.2f}')

정확도 : 82.89


In [29]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic', eval_metric='error', early_stopping_rounds=10)
eval_set = [(X_test, y_test)]

model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'정확도 : {accuracy * 100:.2f}')

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
정확도 : 89.47


In [31]:
model = XGBClassifier(n_estimators=5000, booster='gbtree', objective='binary:logistic', 
                      eval_metric='error', 
                      early_stopping_rounds=100)
eval_set = [(X_test, y_test)]

model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'정확도 : {accuracy * 100:.2f}')

[0]	validation_0-error:0.15789
[1]	validation_0-error:0.10526
[2]	validation_0-error:0.11842
[3]	validation_0-error:0.13158
[4]	validation_0-error:0.11842
[5]	validation_0-error:0.14474
[6]	validation_0-error:0.14474
[7]	validation_0-error:0.14474
[8]	validation_0-error:0.14474
[9]	validation_0-error:0.14474
[10]	validation_0-error:0.14474
[11]	validation_0-error:0.15789
[12]	validation_0-error:0.15789
[13]	validation_0-error:0.17105
[14]	validation_0-error:0.17105
[15]	validation_0-error:0.17105
[16]	validation_0-error:0.15789
[17]	validation_0-error:0.17105
[18]	validation_0-error:0.15789
[19]	validation_0-error:0.17105
[20]	validation_0-error:0.17105
[21]	validation_0-error:0.17105
[22]	validation_0-error:0.18421
[23]	validation_0-error:0.18421
[24]	validation_0-error:0.17105
[25]	validation_0-error:0.18421
[26]	validation_0-error:0.18421
[27]	validation_0-error:0.18421
[28]	validation_0-error:0.18421
[29]	validation_0-error:0.18421
[30]	validation_0-error:0.18421
[31]	validation_0-

## 6.4 하이퍼파라미터 결합

In [33]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'min_child_weight':[1, 2, 3, 4, 5],
                    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5],
                    'max_depth':[1, 2, 3, 4, 5],
                    'n_estimators':[2]})

최상의 매개변수:  {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 2, 'subsample': 0.9}
최상의 점수: 0.81224


In [35]:
grid_search(params={'subsample':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'min_child_weight':[1, 2, 3, 4, 5],
                    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5],
                    'max_depth':[1, 2, 3, 4, 5, None],
                    'n_estimators':[2, 25, 50, 75, 100]}, random=True)

최상의 매개변수:  {'subsample': 0.6, 'n_estimators': 25, 'min_child_weight': 4, 'max_depth': 4, 'learning_rate': 0.5}
최상의 점수: 0.82208
